In [1]:
import sys
!{sys.executable} -m pip install rdflib

In [45]:
import sys
!{sys.executable} -m pip install pandas numpy

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------- ----------------------- 5.2/12.6 MB 26.6 MB/s eta 0:00:01
   ------------------------------------ --- 11.5/12.6 MB 27.8 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 25.5 MB/s eta 0:00:00


In [19]:
#namespaces
maSMP_ns = "https://discovery.biothings.io/view/maSMP/"
maSMPPro_ns = "https://discovery.biothings.io/view/maSMPProfiles/"
schema_ns = "http://schema.org/" #double check whether http or https is used in the source file

## Create an index page for Profiles

In [9]:
from rdflib import Graph
from rdflib import URIRef
from rdflib.namespace import RDF, RDFS, OWL

from pandas import DataFrame

In [20]:
#properties used for filtering triples
rdfs_class = URIRef("http://www.w3.org/2000/01/rdf-schema#Class")
schema_domain = URIRef(schema_ns + "domainIncludes")
schema_range = URIRef(schema_ns + "rangeIncludes")

In [3]:
#load graph
g = Graph()
g.parse("https://raw.githubusercontent.com/zbmed-semtec/maSMPs/refs/heads/main/schema/maSMP_schema_v2/v2.1.0/profiles/maSMP_profiles_v2.jsonld")

<Graph identifier=Neee7e70c9e834b508a746680eae04e5b (<class 'rdflib.graph.Graph'>)>

In [12]:
#Create a generic table listing all the types and their descriptions

table = "<table>\n" + "<tr><th>Profile</th><th>Description</th></tr>\n"

#iterate over all classes in the maSMP namespace to get type and description
for s in g.subjects(object=rdfs_class, unique=True) :
    type_name = str(s).split('/')[-1]
    if maSMPPro_ns in s :
        description = g.value(subject=s, predicate=RDFS.comment)
        table += "<tr><td><a href='./" + type_name[:-len("Profile")] + "'>" + type_name + "</a></td><td>" + description + "</td></tr>\n\n"

table += "</table>\n"

In [13]:
# Create the Profiles page
intro_text = """
<h1>maSMP Profiles</h1>\n
maSMP profiles are recommendations of use corresponding to [maSMP](./Types/index.md) and [schema.org](https://schema.org) types and properties. 
Recommendations relate to the cardinality (one, many) and marginality (minimum, recommended, optional). 
The json.schema corresponding to the maSMP profiles can be found at the [maSMP DDE namespace](https://discovery.biothings.io/ns/maSMPProfiles){:target="_blank"}.
\n\n
To avoid conflict/confusion between types and profiles, all the profile names end with the suffix 'Profile'
\n\n
"""

with open("../docs/Profiles/index.md", 'w') as file:
    file.write(intro_text)
    file.write(table)

## Create individual pages, one per profile

In [15]:
import urllib.request, json

In [35]:
def_value = maSMP_ns
def convert_to_link(url, label=None, md=False) :
    local = True if url.startswith("maSMP:") else False
    
    url = url.replace("maSMP:", "../../Types/")    
    if label is None:
        label = str(url).split('/')[-1]

    if local :
        label = "maSMP:" + label
        if md :
            return "[{}]({})".format(label, url)
        else :
            return "<a href='{}'>{}</a>".format(url, label)
    else :   
        if md :
            return "[{}]({})".format(label, url) + "{:target='_blank'}"
        else :
            return "<a href='{}' target='_blank'>{}</a>".format(url, label)

print(convert_to_link("maSMP:SoftwareRunAction"))

<a href='../../Types/SoftwareRunAction'>maSMP:SoftwareRunAction</a>


In [17]:
origin = "https://raw.githubusercontent.com/zbmed-semtec/maSMPs/refs/heads/main/schema/maSMP_schema_v2/v2.1.0/profiles/maSMP_profiles_v2.jsonld"
with urllib.request.urlopen(origin) as prof_file:
    prof_data = json.load(prof_file)

In [47]:
def get_prop_list(elem, level) :
    lst = []
    for item in elem["$validation"][level] :
        new_row = {"Property": item, "Expected Type": "", "Description": "", "Cardinality": ""}
        new_row["Description"] = elem["$validation"]["properties"][item]["description"]
        new_row["Cardinality"] = elem["$validation"]["properties"][item]["owl:cardinality"]
        lst.append(new_row)
    return lst

In [48]:
lst_min = []
lst_rec= []
lst_opt = []

for elem in prof_data["@graph"] :
    if elem["@type"] == "rdfs:Class" :
        intro_text = "<h1>" + elem["rdfs:label"] + "</h1>\n\n"
        intro_text += elem["rdfs:label"] + " is profile for " + convert_to_link(elem["rdfs:subClassOf"]["@id"]) + "\n\n"
        intro_text += elem["rdfs:comment"] + "\n\n"

        lst_min = get_prop_list(elem, "required")
        lst_rec = get_prop_list(elem, "recommended")
        lst_opt = get_prop_list(elem, "optional")

        with open("../docs/Profiles/" + (elem["rdfs:label"])[:-len("Profile")] + ".md", 'w') as file:
            file.write(intro_text)
            file.write("##Minimum properties\n")
            df_min = DataFrame(lst_min)
            file.write(df_min.to_string())
            file.write("##Recommended properties\n")
            df_min = DataFrame(lst_min)
            file.write(df_min.to_string())
            file.write("##Optional properties\n")
            df_min = DataFrame(lst_min)
            file.write(df_min.to_string())
        

KeyError: 'archivedAt'

In [71]:
#Create individual pages per type
#iterate over all classes in the maSMP namespace to get properties
for s in g.subjects(object=rdfs_class, unique=True) :
    type_name = str(s).split('/')[-1]   
    
    if maSMP_ns in s :
        description = g.value(subject=s, predicate=RDFS.comment)
        parent = g.value(subject=s, predicate=RDFS.subClassOf)

        #info about type, hierarchy, and description
        intro_text = type_name + "is profile for " + convert_to_link(parent) + "\n\n" #add link to parent
        intro_text += description + "\n\n"

        #get all properties that have this type as domain
        get_props_query = "SELECT DISTINCT ?prop ?label ?desc ?range WHERE { "
        get_props_query += "?prop <" + schema_ns + "domainIncludes> <" + s + "> . " 
        get_props_query += "?prop <" + RDFS.label + "> ?label . " 
        get_props_query += "?prop <" + RDFS.comment + "> ?desc . "
        get_props_query += "?prop <" + schema_ns + "rangeIncludes> ?range . " 
        get_props_query += " }"
        qres = g.query(get_props_query)

        table = "<table>\n" + "<tr><th>Property</th><th>Expected Type</th><th>Description</th></tr>"
        df = DataFrame(qres, columns=['prop', 'label', 'desc', 'range'])
        df = df.groupby(['prop']).agg({'label' : 'first', 'desc' : 'first', 'range' : ' or '.join}) 
        for index, row  in df.iterrows() :
            table += "<tr><td>" + row['label'] + "</td>"
            table += "<td>" + row['range'] + "</td>"
            table += "<td>" + row['desc'] + "</td>"
            table += "</tr>\n"
        table += "</table>\n"

        with open("../docs/Types/" + type_name + ".md", 'w') as file:
            file.write(intro_text)
            file.write(table)
        

http://schema.org/Action -  does not look like a valid URI, trying to serialize this will break.
http://schema.org/Action - SoftwareRunAction does not look like a valid URI, trying to serialize this will break.
http://schema.org/Action - SoftwareRunAction

 does not look like a valid URI, trying to serialize this will break.
http://schema.org/Action - SoftwareRunAction

The act of testing an object according to its specifications. For instance, testing a software with a particular testType using a specific testInput and getting a specific testOutput (aka result).

 does not look like a valid URI, trying to serialize this will break.
?prop <http://schema.org/domainIncludes> <https://discovery.biothings.io/view/maSMP/SoftwareRunAction does not look like a valid URI, trying to serialize this will break.
?prop <http://schema.org/domainIncludes> <https://discovery.biothings.io/view/maSMP/SoftwareRunAction> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTIN

SoftwareRunAction
SoftwareTestAction
OutputManagementPlan
SoftwareManagementPlan


In [ ]:
get_props_query = """
SELECT DISTINCT ?id ?range ?desc
WHERE {
  
}
"""

In [ ]:
table = "<table>\n" + "<tr><th>Property</th><th>Expected Type</th><th>Description</th></tr>\n"
        for prop_range in g.objects(subject=s, predicate=schema_range) :
            range_name = str(prop_range).split('/')[-1]
            if maSMP_ns not in prop_range :
                table += "<tr><td><a href='" + prop_range + "'>" + range_name + "</a></td><td>" + description + "</td></tr>\n\n"
            table += "<tr><td><a href='./" + sufix + "'>" + sufix + "</a></td><td>" + description + "</td></tr>\n\n"
        table += "</table>\n"